In [3]:
# import libraries
import os
from dotenv import load_dotenv
from groq import Groq

# load environment variables
load_dotenv()

# create client calling Groq class
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

# create a query
response = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "I want to increase my fitness level as a cricketer, what to do, i do not have any equipment",
        }
    ],
    model="llama3-8b-8192",
)

# print the response
print(response.choices[0].message.content)

As a cricketer, improving your fitness is crucial to enhance your performance on the field. There are many ways to do so without any equipment, and I'm happy to share some exercises and drills to help you get started:

**Warm-up exercises (10-15 minutes)**

1. Jogging: Start with a light jog to get your heart rate up and warm up your muscles.
2. Leg swings: Stand with your feet hip-width apart and swing one leg forward and backward, then switch to the other leg.
3. Arm circles: Hold your arms straight out to the sides and make small circles with your hands.
4. Neck stretches: Slowly tilt your head to the side, bringing your ear towards your shoulder. Repeat on the other side.
5. Shoulder rolls: Roll your shoulders forward and backward in a circular motion.

**Bodyweight exercises (30-40 minutes)**

1. Squats: Stand with your feet shoulder-width apart and squat down, keeping your back straight and your knees behind your toes.
2. Push-ups: Start in a plank position and lower your body un

In [ ]:
# In this section we use a simple inbuilt search engine
# Download the necessary files using wget
import wget

url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/minsearch.py'
wget.download(url)
url = 'https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/main/01-intro/documents.json'
wget.download(url)

In [11]:
import os
import json
from dotenv import load_dotenv
import minsearch
from groq import Groq
load_dotenv()

with open("documents.json",'rt') as f:
  docs_raw = json.load(f)
documents = []
for course_dict in docs_raw:
  for doc in course_dict['documents']:
    doc['course'] = course_dict['course']
    documents.append(doc)

index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)


query = 'the course has already started, can I still enroll?'

index.fit(documents)

boost = {'question': 3.0, 'section': 0.5}

results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.
    QUESTION: {question}
    CONTEXT: 
    {context}
    """.strip()

    context = ""
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

prompt = build_prompt(query,results)

#client = OpenAI(api_key=os.getenv('OPEN_API_KEY'))
client = Groq(api_key=os.getenv('GROQ_API_KEY'))

response = client.chat.completions.create(
    model="llama3-8b-8192",
    messages = [{"role":"user","content":prompt}])

print(response.choices[0].message.content)

Based on the context, my answer to the question "The course has already started, can I still enroll?" is:

No, the course has started, so it is not possible to register for the course at this point. However, as stated in another answer, even if you don't register, you're still eligible to submit the homeworks.
